In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
from sklearnex import patch_sklearn #Improves sklearn alghoritms performance
patch_sklearn()
import sklearn
print('scikit-learn version\n', sklearn.__version__)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV
from aux_functions.binary_classifiers import binary_data_sampling
import pickle
from tqdm.notebook import tqdm
import os

scikit-learn version
 1.0.2


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
#Set seed
seed=41

In [3]:
#Classification Level -> Superclass
classif_level = 1

# Load Data

In [4]:
feature_names = pickle.load(open('Dataset/feature_names.pkl', 'rb'))
X_train = pickle.load(open(f'Dataset/X_train.pkl', 'rb'))
X_test = pickle.load(open(f'Dataset/X_test.pkl', 'rb'))
y_train = pickle.load(open(f'Dataset/y_train.pkl', 'rb'))
y_test = pickle.load(open(f'Dataset/y_test.pkl', 'rb'))

In [5]:
len(feature_names)

133

In [6]:
X_train_org = X_train[y_train[:,classif_level-1] == 'Organic compounds', :]
X_test_org = X_test[y_test[:,classif_level-1] == 'Organic compounds', :]
y_train_org = y_train[y_train[:,classif_level-1] == 'Organic compounds'][:, classif_level]
y_test_org = y_test[y_test[:,classif_level-1] == 'Organic compounds'][:, classif_level]

In [7]:
#all classes
org_sclasses = np.unique(y_train_org)

In [8]:
for i, sclass_ in enumerate(org_sclasses):
    print(i, sclass_)

0 Acetylides
1 Alkaloids and derivatives
2 Allenes
3 Benzenoids
4 Hydrocarbon derivatives
5 Hydrocarbons
6 Lignans, neolignans and related compounds
7 Lipids and lipid-like molecules
8 Nucleosides, nucleotides, and analogues
9 Organic 1,3-dipolar compounds
10 Organic Polymers
11 Organic acids and derivatives
12 Organic compounds - Unspecified
13 Organic nitrogen compounds
14 Organic oxygen compounds
15 Organic salts
16 Organohalogen compounds
17 Organoheterocyclic compounds
18 Organometallic compounds
19 Organophosphorus compounds
20 Organosulfur compounds
21 Phenylpropanoids and polyketides


In [24]:
results = []
for f_sel in [False, True]:
    for samp_strat in [None, 'stratf', 'ObyO']:
        for mult_factor in [1.3, 1.5, 2, 3, 4, 5, 6]:
            if samp_strat is None:
                mult_factor = None
            for i, sclass_ in enumerate(org_sclasses):
                print(f_sel)
                print(samp_strat)
                print(mult_factor)
                print(sclass_)
                
                if f_sel is True:
                    sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass_binary'][sclass_]['selected features']
                    f_index = [feature_names.index(feature) for feature in sel_f]
                else:
                    sel_f = None
                    f_index = None
                    
                X_train, y_train, X_test, y_test, train_pos_size, train_neg_size = binary_data_sampling(X_train_org, y_train_org, 
                                                                                                        X_test_org, y_test_org, 
                                                                                                        sclass_, samp_strat, 
                                                                                                        mult_factor, seed=seed, 
                                                                                                        feature_selection=f_sel, 
                                                                                                        features_index=f_index)
                if f_sel is True:
                    grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                            'criterion' : ['gini', 'entropy'],
                            'max_features' : range(1, np.shape(X_train)[1]+1)}
                else:
                    grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                            'criterion' : ['gini', 'entropy'],
                            'max_features' : [1, 2, 3, 5, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 133]}
                '''
                estimator = RandomForestClassifier(random_state=seed, n_jobs=-1)
                gs = GridSearchCV(estimator, param_grid=grid, scoring='balanced_accuracy', cv=3, verbose=100, 
                                  pre_dispatch=True, error_score='raise', return_train_score=True, n_jobs=-1)
                gs.fit(X_train, y_train)
                pickle.dump(gs, open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'wb'))
                '''
                gs = pickle.load(open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'rb'))
                
                print(gs.best_score_)
                estimator = gs.best_estimator_
                cval_results = gs.cv_results_
                print('Pos size=',train_pos_size)
                print('Neg size=',train_neg_size)
                score_train = cval_results['mean_train_score'][gs.best_index_]
                score_val = cval_results['mean_test_score'][gs.best_index_]
                print('Balanced accuracy (cv-train) ->', score_train)
                print('Balanced accuracy (cv-val) ->', score_val)
                ##Test binary RF
                y_pred = estimator.predict(X_test)
                score_test = balanced_accuracy_score(y_test, y_pred)
                print('Balanced accuracy (test) ->', score_test)
                results.append({'Superclass': sclass_, 'Feature selection':f_sel, 'Sampling strategy': str(samp_strat), 
                                'Multiplying factor': mult_factor,
                                'Train positive size': train_pos_size, 'Train negative size':train_neg_size,
                                'Balanced accuracy (cv-train)':score_train,
                                'Balanced accuracy (cv-val)':score_val,
                                'Balanced accuracy (test)':score_test})
            if samp_strat is None:
                break
results = pd.DataFrame(results)
results.replace({True:'Yes', False:'No'}).to_csv('Results/GS_Binary_RF_(Superclass-Org compounds).csv')
results.to_pickle('Results/GS_Binary_RF_(Superclass-Org compounds).pkl')

False
None
None
Acetylides
0.8577563483030307
Pos size= 14
Neg size= 193246
Balanced accuracy (cv-train) -> 0.9995083987185103
Balanced accuracy (cv-val) -> 0.8577563483030307
Balanced accuracy (test) -> 0.7136553297511482
False
None
None
Alkaloids and derivatives
0.9175170558179175
Pos size= 1092
Neg size= 192168
Balanced accuracy (cv-train) -> 0.9822030723117271
Balanced accuracy (cv-val) -> 0.9175170558179175
Balanced accuracy (test) -> 0.9302226668673472
False
None
None
Allenes
0.9432465316666683
Pos size= 7
Neg size= 193253
Balanced accuracy (cv-train) -> 0.9988266677095506
Balanced accuracy (cv-val) -> 0.9432465316666683
Balanced accuracy (test) -> 0.9979986132123047
False
None
None
Benzenoids
0.9031070596816865
Pos size= 10146
Neg size= 183114
Balanced accuracy (cv-train) -> 0.9437549590132627
Balanced accuracy (cv-val) -> 0.9031070596816865
Balanced accuracy (test) -> 0.9012798588902899
False
None
None
Hydrocarbon derivatives
0.8824329872015291
Pos size= 35
Neg size= 193225
Bal

0.9338461538461539
Pos size= 1200
Neg size= 1560
Balanced accuracy (cv-train) -> 0.9974999999999999
Balanced accuracy (cv-val) -> 0.9338461538461539
Balanced accuracy (test) -> 0.9373578501318804
False
stratf
1.3
Organic oxygen compounds
0.8891617356354952
Pos size= 8094
Neg size= 10522
Balanced accuracy (cv-train) -> 0.9834609272561323
Balanced accuracy (cv-val) -> 0.8891617356354952
Balanced accuracy (test) -> 0.8966937646140889
False
stratf
1.3
Organic salts
1.0
Pos size= 43
Neg size= 65
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 1.0
Balanced accuracy (test) -> 1.0
False
stratf
1.3
Organohalogen compounds
0.9923977460409987
Pos size= 218
Neg size= 283
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 0.9923977460409987
Balanced accuracy (test) -> 0.9883623468729852
False
stratf
1.3
Organoheterocyclic compounds
0.8979891786234125
Pos size= 14296
Neg size= 18584
Balanced accuracy (cv-train) -> 0.9687690201074594
Balanced accuracy (cv-val) -> 0.89

0.9698765068723395
Pos size= 133382
Neg size= 59878
Balanced accuracy (cv-train) -> 0.9814733301532205
Balanced accuracy (cv-val) -> 0.9698765068723395
Balanced accuracy (test) -> 0.9706897580521074
False
stratf
2
Nucleosides, nucleotides, and analogues
0.9740549633507743
Pos size= 935
Neg size= 1870
Balanced accuracy (cv-train) -> 0.9990643140111978
Balanced accuracy (cv-val) -> 0.9740549633507743
Balanced accuracy (test) -> 0.9760934544520468
False
stratf
2
Organic 1,3-dipolar compounds
0.9469696969696969
Pos size= 66
Neg size= 132
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 0.9469696969696969
Balanced accuracy (test) -> 0.9318181818181819
False
stratf
2
Organic Polymers
0.9974006116207952
Pos size= 1635
Neg size= 3270
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 0.9974006116207952
Balanced accuracy (test) -> 0.995080853840842
False
stratf
2
Organic acids and derivatives
0.926993512359366
Pos size= 12177
Neg size= 24354
Balanced accuracy (cv

0.9799110508065733
Pos size= 14
Neg size= 200
Balanced accuracy (cv-train) -> 0.9887498597239368
Balanced accuracy (cv-val) -> 0.9799110508065733
Balanced accuracy (test) -> 0.985
False
stratf
4
Alkaloids and derivatives
0.9423076923076922
Pos size= 1092
Neg size= 4368
Balanced accuracy (cv-train) -> 0.9791666666666666
Balanced accuracy (cv-val) -> 0.9423076923076922
Balanced accuracy (test) -> 0.9418178423066691
False
stratf
4
Allenes
1.0
Pos size= 7
Neg size= 200
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 1.0
Balanced accuracy (test) -> 1.0
False
stratf
4
Benzenoids
0.9062191996846048
Pos size= 10146
Neg size= 40584
Balanced accuracy (cv-train) -> 0.956405233589592
Balanced accuracy (cv-val) -> 0.9062191996846048
Balanced accuracy (test) -> 0.9036615266952346
False
stratf
4
Hydrocarbon derivatives
0.9235828433589628
Pos size= 35
Neg size= 200
Balanced accuracy (cv-train) -> 0.9725058915946582
Balanced accuracy (cv-val) -> 0.9235828433589628
Balanced accuracy (t

0.8926612305411415
Pos size= 8094
Neg size= 40470
Balanced accuracy (cv-train) -> 0.973023227081789
Balanced accuracy (cv-val) -> 0.8926612305411415
Balanced accuracy (test) -> 0.9009047976293603
False
stratf
5
Organic salts
0.9979919678714859
Pos size= 43
Neg size= 250
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 0.9979919678714859
Balanced accuracy (test) -> 0.9721904761904762
False
stratf
5
Organohalogen compounds
0.991272371487191
Pos size= 218
Neg size= 1090
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy (cv-val) -> 0.991272371487191
Balanced accuracy (test) -> 0.9781484570475396
False
stratf
5
Organoheterocyclic compounds
0.9024484252883495
Pos size= 14296
Neg size= 71480
Balanced accuracy (cv-train) -> 0.9472089826129784
Balanced accuracy (cv-val) -> 0.9024484252883495
Balanced accuracy (test) -> 0.907870548738062
False
stratf
5
Organometallic compounds
0.9858874975154045
Pos size= 77
Neg size= 385
Balanced accuracy (cv-train) -> 1.0
Balanced accuracy

...

0.98
Pos size= 27
Neg size= 300
Balanced accuracy (cv-train) -> 0.9975
Balanced accuracy (cv-val) -> 0.98
Balanced accuracy (test) -> 0.98
True
ObyO
6
Organosulfur compounds


/home/hsilva/Tese/ML/aux_functions/sampling.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  class_sampled = pd.Series()
/home/hsilva/Tese/ML/aux_functions/sampling.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  class_sampled = pd.Series()


0.9591258183649488
Pos size= 481
Neg size= 2886
Balanced accuracy (cv-train) -> 0.9827616077616077
Balanced accuracy (cv-val) -> 0.9591258183649488
Balanced accuracy (test) -> 0.965848336061102
True
ObyO
6
Phenylpropanoids and polyketides


/home/hsilva/Tese/ML/aux_functions/sampling.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  class_sampled = pd.Series()
/home/hsilva/Tese/ML/aux_functions/sampling.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  class_sampled = pd.Series()


0.8788904727963885
Pos size= 8418
Neg size= 50508
Balanced accuracy (cv-train) -> 0.9679258731290092
Balanced accuracy (cv-val) -> 0.8788904727963885
Balanced accuracy (test) -> 0.9211913757955783


# Prediction

In [27]:
df = pd.read_pickle('Results/GS_Binary_RF_(Superclass-Org compounds).pkl')

### select classifiers from all

In [28]:
cfs = {}
for i, sclass_ in enumerate(org_sclasses):
    c_results = df[df['Superclass']==sclass_]
    max_ = c_results['Balanced accuracy (cv-val)'].idxmax()
    row = c_results.loc[max_]
    samp_strat = row['Sampling strategy']
    mult_factor = row['Multiplying factor']
    f_sel = row['Feature selection']
    if samp_strat == 'None':
        mult_factor = None
    if mult_factor in [1, 2, 3, 4, 5, 6]:
        mult_factor = int(mult_factor)
    cfs[sclass_] = (pickle.load(open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'rb')).best_estimator_, f_sel)

In [85]:
new_pred = {}
index = np.where([True if i in org_sclasses else False for i in y_test_org])[0]
y_test = y_test_org[index]
for sclass_ in tqdm(cfs):
    X_test_org_ = X_test_org
    cf = cfs[sclass_][0]
    f_sel = cfs[sclass_][1]
    if f_sel:
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass_binary'][sclass_]['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_test_org_= X_test_org_[:, f_index]
    new_pred[sclass_] = cf.predict_proba(X_test_org_[index])[:, 1]

  0%|          | 0/22 [00:00<?, ?it/s]

In [86]:
new_pred = pd.DataFrame(new_pred)

In [87]:
y_pred = new_pred.idxmax(axis=1).values

In [88]:
f1_score(y_test, y_pred, average='macro')

0.42658235803650213

In [89]:
f1_score(y_test, y_pred, average='micro')

0.83804511120216

In [90]:
classification_report(y_test, y_pred, output_dict=True)

{'Acetylides': {'precision': 0.006711409395973154,
  'recall': 0.14285714285714285,
  'f1-score': 0.012820512820512822,
  'support': 7},
 'Alkaloids and derivatives': {'precision': 0.18283220174587778,
  'recall': 0.702048417132216,
  'f1-score': 0.29011158137745285,
  'support': 537},
 'Allenes': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3},
 'Benzenoids': {'precision': 0.46378072656080055,
  'recall': 0.4268268268268268,
  'f1-score': 0.44453711426188486,
  'support': 4995},
 'Hydrocarbon derivatives': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 19},
 'Hydrocarbons': {'precision': 0.5983263598326359,
  'recall': 0.9438943894389439,
  'f1-score': 0.732394366197183,
  'support': 303},
 'Lignans, neolignans and related compounds': {'precision': 0.07672849915682968,
  'recall': 0.6363636363636364,
  'f1-score': 0.13694507148231752,
  'support': 143},
 'Lipids and lipid-like molecules': {'precision': 0.9955559864901989,
  'recall': 0.93778636993

### select classifiers from no sampling

In [91]:
cfs = {}
for i, sclass_ in enumerate(org_sclasses):
    c_results = df[(df['Superclass']==sclass_) & (df['Sampling strategy'].isnull())]
    max_ = c_results['Balanced accuracy (cv-val)'].idxmax()
    row = c_results.loc[max_]
    samp_strat = None
    mult_factor = None
    f_sel = row['Feature selection']
    cfs[sclass_] = (pickle.load(open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'rb')).best_estimator_, f_sel)

In [92]:
new_pred = {}
index = np.where([True if i in org_sclasses else False for i in y_test_org])[0]
y_test = y_test_org[index]
for sclass_ in tqdm(cfs):
    X_test_org_ = X_test_org
    cf = cfs[sclass_][0]
    f_sel = cfs[sclass_][1]
    if f_sel:
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass_binary'][sclass_]['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_test_org_= X_test_org_[:, f_index]
    new_pred[sclass_] = cf.predict_proba(X_test_org_[index])[:, 1]

  0%|          | 0/22 [00:00<?, ?it/s]

In [93]:
new_pred = pd.DataFrame(new_pred)

In [94]:
y_pred = new_pred.idxmax(axis=1).values

In [95]:
f1_score(y_test, y_pred, average='macro')

0.40601702595448336

In [96]:
f1_score(y_test, y_pred, average='micro')

0.8365007826699025

### select classifiers from stratf sampling

In [97]:
cfs = {}
for i, sclass_ in enumerate(org_sclasses):
    c_results = df[(df['Superclass']==sclass_) & (df['Sampling strategy']=='stratf')]
    max_ = c_results['Balanced accuracy (cv-val)'].idxmax()
    row = c_results.loc[max_]
    samp_strat = 'stratf'
    mult_factor = row['Multiplying factor']
    f_sel = row['Feature selection']
    if mult_factor in [1, 2, 3, 4, 5, 6]:
        mult_factor = int(mult_factor)
    cfs[sclass_] = (pickle.load(open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'rb')).best_estimator_, f_sel)

In [98]:
new_pred = {}
index = np.where([True if i in org_sclasses else False for i in y_test_org])[0]
y_test = y_test_org[index]
for sclass_ in tqdm(cfs):
    X_test_org_ = X_test_org
    cf = cfs[sclass_][0]
    f_sel = cfs[sclass_][1]
    if f_sel:
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass_binary'][sclass_]['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_test_org_= X_test_org_[:, f_index]
    new_pred[sclass_] = cf.predict_proba(X_test_org_[index])[:, 1]

  0%|          | 0/22 [00:00<?, ?it/s]

In [99]:
new_pred = pd.DataFrame(new_pred)

In [100]:
y_pred = new_pred.idxmax(axis=1).values

In [101]:
f1_score(y_test, y_pred, average='macro')

0.42658235803650213

In [102]:
f1_score(y_test, y_pred, average='micro')

0.83804511120216

### select classifiers from ObyO sampling

In [103]:
cfs = {}
for i, sclass_ in enumerate(org_sclasses):
    c_results = df[(df['Superclass']==sclass_) & (df['Sampling strategy']=='ObyO')]
    max_ = c_results['Balanced accuracy (cv-val)'].idxmax()
    row = c_results.loc[max_]
    samp_strat = 'ObyO'
    mult_factor = row['Multiplying factor']
    f_sel = row['Feature selection']
    if mult_factor in [1, 2, 3, 4, 5, 6]:
        mult_factor = int(mult_factor)
    cfs[sclass_] = (pickle.load(open(f'Models/Superclass/Org_RF_Binary/Feature_selection={f_sel}/Sampl_strategy={samp_strat}/{i}_multfactor={mult_factor}.pkl', 'rb')).best_estimator_, f_sel)

In [104]:
new_pred = {}
index = np.where([True if i in org_sclasses else False for i in y_test_org])[0]
y_test = y_test_org[index]
for sclass_ in tqdm(cfs):
    X_test_org_ = X_test_org
    cf = cfs[sclass_][0]
    f_sel = cfs[sclass_][1]
    if f_sel:
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass_binary'][sclass_]['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_test_org_= X_test_org_[:, f_index]
    new_pred[sclass_] = cf.predict_proba(X_test_org_[index])[:, 1]

  0%|          | 0/22 [00:00<?, ?it/s]

In [105]:
new_pred = pd.DataFrame(new_pred)

In [106]:
y_pred = new_pred.idxmax(axis=1).values

In [107]:
f1_score(y_test, y_pred, average='macro')

0.40171567621278226

In [108]:
f1_score(y_test, y_pred, average='micro')

0.8201750239003225

### Prediction on test using the multiclass RF

In [109]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Superclass/1_RF_{f_sel}_feat.pkl', 'rb'))
    print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

all
f1_macro_test -> 0.5202564530455621
f1_macro_train -> 0.8395686699909547
f1_micro_test -> 0.8776570423263997
f1_micro_train -> 0.9514979819931698
sel
f1_macro_test -> 0.5258085250500398
f1_macro_train -> 0.826792738664746
f1_micro_test -> 0.8783193625168167
f1_micro_train -> 0.9503233985304771


In [110]:
#Best one is with feature selection (f1 macro test)
gs = pickle.load(open(f'Models/Superclass/1_RF_sel_feat.pkl', 'rb'))
sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass']['Organic compounds']['selected features']
f_index = [feature_names.index(feature) for feature in sel_f]
y_pred = gs.best_estimator_.predict(X_test_org[:, f_index])

In [111]:
f1_score(y_test_org, y_pred, average='macro')

0.5186876912737598

In [112]:
f1_score(y_test_org, y_pred, average='micro')

0.884427495351256